In [1]:
from langchain_community.document_loaders import DirectoryLoader, JSONLoader

def metadata_func(record, metadata):
    metadata.update({
        "title": record.get("title", ""),
        "section": record.get("section", ""),
        "chunk_index": record.get("chunk_index", ""),
    })
    return metadata

loader = DirectoryLoader(
    "./data",
    glob="attack_on_Titan_Namu_part1.jsonl",
    loader_cls=JSONLoader,
    loader_kwargs={
        "jq_schema": ".",
        "json_lines": True,
        "content_key": "text",
        "metadata_func": metadata_func,
    },
)

document_list = loader.load()


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000,
    chunk_overlap=200,
)
MAX_TOKENS = 3500

def is_table_or_quote(text: str) -> bool:
    s = text.strip()
    return (
        (s.startswith("[TABLE]") and s.endswith("[/TABLE]")) or
        (s.startswith("[QUOTE]") and s.endswith("[/QUOTE]"))
    )

split_docs = []
for doc in document_list:
    tokens = splitter._length_function(doc.page_content)
    if is_table_or_quote(doc.page_content):
        if tokens > MAX_TOKENS:
            split_docs.extend(splitter.split_documents([doc]))
        else:
            split_docs.append(doc)
        continue

    if tokens > MAX_TOKENS:
        split_docs.extend(splitter.split_documents([doc]))
    else:
        split_docs.append(doc)




In [6]:
max_doc = max(split_docs, key=lambda d: splitter._length_function(d.page_content))
splitter._length_function(max_doc.page_content), max_doc.metadata


(3458,
 {'source': '/mnt/e/one_piece/data/attack_on_Titan_Namu_part1.jsonl',
  'seq_num': 10277,
  'title': '진격의 궤적 - 나무위키',
  'section': '3.8. 양날개의 빛',
  'chunk_index': '20'})

In [ ]:
# from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings

# # 환경변수를 불러옴
# load_dotenv()

# # OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
# embedding = OpenAIEmbeddings(model='text-embedding-3-small')

In [7]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = UpstageEmbeddings(model='solar-embedding-1-large', embed_batch_size=1)

In [ ]:
from langchain_chroma import Chroma

# 데이터를 처음 저장할 때 
database = Chroma.from_documents(documents=split_docs, embedding=embedding, collection_name='AoT', persist_directory="./AoT")


In [ ]:
from langchain_chroma import Chroma
from tqdm import tqdm

db = Chroma(
    collection_name="AoT",
    embedding_function=embedding,
    persist_directory="./AoT",
)

batch_size = 64
total = len(split_docs)

for i in tqdm(range(0, total, batch_size)):
    batch = split_docs[i : i + batch_size]
    db.add_documents(batch)




100%|██████████| 217/217 [2:17:42<00:00, 38.08s/it]  


AttributeError: 'Chroma' object has no attribute 'persist'

In [ ]:
# import shutil

# persist_directory = "AoT"  # ✅ ChromaDB 저장 경로
# shutil.rmtree(persist_directory)  # ✅ 기존 ChromaDB 폴더 삭제
# print(f"✅ 기존 ChromaDB ({persist_directory}) 삭제 완료!")


✅ 기존 ChromaDB (AoT) 삭제 완료!


In [11]:
from langchain_chroma import Chroma

database = Chroma(
    collection_name="AoT",
    persist_directory="./AoT",
    embedding_function=embedding,
)

In [19]:
from langchain_upstage import ChatUpstage
from langchain_core.messages import SystemMessage, HumanMessage

retriever = database.as_retriever(search_kwargs={"k": 5})

query = "원작에서 애니 레온하트는 아르민을 좋아해?"
docs = retriever.invoke(query)
context = "\n\n".join([d.page_content for d in docs])

llm = ChatUpstage(
    model="solar-pro2",  # 계정에 있는 Solar chat 모델명으로 변경
    temperature=0,
)

messages = [
    SystemMessage(content="You are a helpful assistant. Use the context to answer."),
    HumanMessage(content=f"질문: {query}\n\n컨텍스트:\n{context}"),
]

response = llm.invoke(messages)
print(response.content)


원작에서 애니 레온하트가 아르민을 좋아한다는 직접적인 언급이나 명확한 증거는 없습니다. 다만, 컨텍스트에 따르면 **2차 창작에서 애니와 아르민이 커플로 자주 엮이는 경우**가 있으며, 특히 아르민이 애니를 도발하자 애니가 침착함을 잃는 장면이나, 아르민의 거짓말에 분노하는 반응 등이 팬들 사이에서 호감의 가능성으로 해석되기도 합니다.  

하지만 원작 내에서 애니의 감정이나 호감은 명확히 드러나지 않습니다. 오히려 애니의 관심은 주로 **엘런**이나 **마레 전사 동료들**(라이너, 베르톨트)과의 관계에 집중되어 있으며, 아르민과의 관계는 전략적 협력이나 복잡한 과거사에 더 가깝습니다.  

### 주요 근거:
1. **아르민의 도발에 대한 반응**: 아르민이 애니를 고문을 받는다는 거짓말로 도발하자 애니가 분노한 것은 사실이지만, 이는 단순한 분노일 뿐 호감과는 무관할 수 있습니다.  
2. **2차 창작에서의 인기**: 팬들 사이에서 애니×아르민 커플링이 자주 등장하지만, 이는 원작자의 공식 설정이 아닙니다.  
3. **애니와 아르민의 관계**: 원작에서 두 사람은 서로를 이용하거나 대립하는 관계에 가깝습니다. 예를 들어, 아르민이 애니를 속여 엘런의 탈출을 도운 사건은 애니가 아르민의 계략을 이미 눈치채고 있었음을 보여줍니다.  

따라서 **"원작에서 애니가 아르민을 좋아한다는 공식적 근거는 없으며, 팬들의 추측이나 2차 창작에서의 인기 때문일 가능성이 높습니다**."


In [ ]:
from langchain_openai import ChatOpenAI

retriever = database.as_retriever(search_kwargs={"k": 3})
llm = ChatOpenAI(model="gpt-5")

query = "애니 레온하트는 누굴"
docs = retriever.invoke(query)

context = "\n\n".join([d.page_content for d in docs])

prompt = f"""You are a helpful assistant. Use the context to answer.
질문: {query}

컨텍스트:
{context}
"""

response = llm.invoke(prompt)
response.content


KeyboardInterrupt: 

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

llm = ChatOpenAI(model="gpt-4o")

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the context to answer."),
    ("human", "질문: {input}\n\n컨텍스트:\n{context}")
])

combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

ai_message = retrieval_chain.invoke({"input": query})
ai_message["answer"]


In [ ]:
# RAG: reduce hallucination by stricter retrieval + prompt
from langchain_openai import ChatOpenAI

# retriever with more candidates + MMR for diversity
retriever = database.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "fetch_k": 24}
)

query = "아홉거인에 대해 자세하게 설명해줘"
docs = retriever.invoke(query)

# if retrieval is weak, fail fast
if len(docs) < 2:
    raise ValueError("컨텍스트가 부족합니다. 질문을 구체화하거나 k를 늘려주세요.")

context = "\n\n".join([d.page_content for d in docs])

prompt = f"""You are a helpful assistant.
Use ONLY the provided context.
If the answer is not in the context, say "정보 부족".


질문: {query}

컨텍스트:
{context}
"""

llm = ChatOpenAI(model="gpt-5-mini")
response = llm.invoke(prompt)
response.content


In [ ]:
# Filter category/low-signal docs to reduce hallucination
def filter_docs(docs, min_len=50):
    filtered = []
    for d in docs:
        title = (d.metadata.get('title') or '')
        url = (d.metadata.get('url') or '')
        text = (d.page_content or '').strip()
        if title.startswith('분류:'):
            continue
        if '/w/%EB%B6%84%EB%A5%98:' in url or '/w/분류:' in url:
            continue
        if len(text) < min_len:
            continue
        filtered.append(d)
    return filtered

retriever = database.as_retriever(
    search_type='mmr',
    search_kwargs={'k': 12, 'fetch_k': 40}
)

docs = retriever.invoke(query)
docs = filter_docs(docs)
if len(docs) < 2:
    raise ValueError('정보 부족: 관련 본문 문서를 찾지 못했습니다.')

context = '\n\n'.join([d.page_content for d in docs])

prompt = f"""You are a helpful assistant.
Use ONLY the provided context.
If the answer is not in the context, say \"정보 부족\".


질문: {query}

컨텍스트:
{context}
"""

llm = ChatOpenAI(model='gpt-4o-mini')
response = llm.invoke(prompt)
response.content
